In [ ]:
# Install a pip package in the current Jupyter kernel - REMOVE IN PROD
#import sys
#!{sys.executable} -m pip install gtfparse

In [ ]:
# Import libs
import os.path
from os import path
import gzip
import pandas as pd
import urllib.request
from gtfparse import read_gtf
from biomart import BiomartServer

In [ ]:
# Setup
data_path = '../data'
gtf_path_gz = data_path + '/Mus_musculus.GRCm38.99.gtf.gz'
gtf_path = data_path + '/Mus_musculus.GRCm38.99.gtf'
tpm_yang_path_gz = data_path + '/GSE90848_Ana6_basal_hair_bulb_TPM.txt.gz'
tpm_yang_path = data_path + '/GSE90848_Ana6_basal_hair_bulb_TPM.txt'
gene_tsv_path = data_path + '/gene_names.tsv'

In [ ]:
df_gtf = read_gtf(gtf_path)

In [ ]:
df_gtf

In [ ]:
df_gtf_filt = df_gtf[(df_gtf.feature=='exon') | (df_gtf.feature=='three_prime_utr') | (df_gtf.feature=='five_prime_utr')]
df_gtf_filt.shape